# 要約 
このJupyter Notebookは、LMSYS - Chatbot Arenaコンペティションにおいて、チャットボットの応答に対するユーザーの好みを予測するモデルを構築することを目的としています。

**取り組む問題**
コンペティションは、与えられたプロンプトに対して生成された2つの異なる応答から、どちらが好まれるかを予測することです。具体的には、ユーザーが選好を示すデータセットを基に、大規模言語モデル（LLM）の応答を評価する選好モデルを構築します。

**手法とライブラリ**
- **データの準備**:
  - `pandas`ライブラリを使用して、トレーニングデータとテストデータを読み込み、トレーニングデータを50%にランダムサンプリングしています。
  
- **特徴抽出**:
  - プロンプトと二つの応答（`response_a` と `response_b`）を結合し、それぞれのテキストを準備します。次に、`tensorflow.keras`ライブラリを用いて、テキストをトークン化し、シーケンスに変換し、パディング処理を行います。

- **モデルの定義**:
  - LSTM（Long Short-Term Memory）モデルが`tensorflow.keras`を使用して構築されています。モデルには埋め込み層、LSTM層、ドロップアウト層、出力層が含まれます。これにより、時系列データであるテキストの特徴を適切に捉えます。

- **モデルの学習**:
  - 各応答に対して別々のモデルが訓練され、トレーニングデータを使用して好みの予測を学習します。5エポックでの訓練が行われ、データの70%をバリデーションに使用しています。

- **予測と提出ファイルの作成**:
  - テストデータでの各モデルによる予測が行われ、最終的な提出ファイルが準備されます。引き分けの確率は単純に均一分布（1/3）で計算されています。

このノートブックは、ユーザーの好みを予測するために、機械学習の手法であるLSTMを用いたモデルを訓練し、データの前処理からモデルの予測、最終的な提出データの準備までを一貫して行っています。

---


# 用語概説 
以下に、Jupyter Notebookの内容に関連して、機械学習・深層学習の初心者がつまずきそうな専門用語の簡易解説をまとめました。

1. **サンプリング (Sampling)**:
   サンプリングは、全体のデータセットから一部のデータを無作為に選ぶプロセスです。ここでは、データを減らすために全体の50%をサンプリングし、新たなデータセットを作成しています。サンプリングによって、計算負荷を軽減したり、データを管理しやすくすることが可能です。

2. **トークナイザー (Tokenizer)**:
   トークナイザーは、テキストデータを単語や句などの「トークン」に分割するためのツールです。このプロセスにより、自然言語を数値データとしてモデルに入力できる形式に変換することができます。ノートブックでは、プロンプトと応答を結合したテキストに対してトークナイザーが適用されています。

3. **パディング (Padding)**:
   パディングは、シーケンスの長さを揃えるために、短いシーケンスの末尾にゼロなどの値を追加する処理です。これにより、ニューラルネットワークが同じ入力長で処理できるようになります。ここでは、すべてのシーケンスを最大長に合わせてパディングしています。

4. **LSTM (Long Short-Term Memory)**:
   LSTMは、時系列データを扱うために設計された特殊なリカレントニューラルネットワーク（RNN）の一種です。長期依存関係を学習する能力が高く、テキストやシーケンスデータの処理に適しています。このノートブックでは、LSTMを使用して比較的長いテキストの相互関係を学習しています。

5. **ドロップアウト (Dropout)**:
   ドロップアウトは、過学習を防ぐための正則化手法です。トレーニング中に、一部のニューロンをランダムに無効化することによって、モデルが特定のニューロンに依存しすぎることを防ぎます。ここでは、50%の確率でドロップアウトを行う設定がされています。

6. **シグモイド活性化関数 (Sigmoid Activation Function)**:
   シグモイド関数は、出力を0と1の間に制限する活性化関数です。二値分類問題に適しており、このノートブックではモデルの出力層で使用され、各モデルの勝者確率を求めるために利用されます。

7. **バッチサイズ (Batch Size)**:
   バッチサイズは、モデルのトレーニング中に一度に処理するサンプルの数を指します。小さすぎるとトレーニングが不安定になり、大きすぎると計算リソースを無駄にしてしまう亜交流があるため慎重に選ぶ必要があります。このノートブックでは、64のバッチサイズが使用されています。

8. **検証分割 (Validation Split)**:
   検証分割は、トレーニングデータの一部を検証データとして使用し、モデルの性能を評価するために取った手法です。この方法によって、モデルがトレーニングデータに過度に適合しているかどうかを確認することができます。ここでは70%をトレーニングに使い、残りの30%を検証に利用しています。

9. **フラット化 (Flatten)**:
   フラット化は、多次元の配列を1次元に変換する処理です。このノートブックでは、LSTMモデルの予測結果を一列に並べるために使用されています。これは、通常のデータ処理に必要な形式に変換するためのステップです。

10. **提出ファイル (Submission File)**:
    提出ファイルは、コンペティションなどで投稿するための結果データが含まれたCSVファイルです。このファイルには、テストデータに対してモデルが予測した出力が含まれ、コンペティションの評価のために必要です。ノートブックの最後でこのファイルが作成されています。

これらの用語は、ノートブックの内容に深く関連しており、特に初心者が理解する際に重要なポイントです。

---


In [ ]:
# データを50%に削減する

import pandas as pd

# train.csvファイルを読み込む
train_data_path = '/kaggle/input/lmsys-chatbot-arena/train.csv'  # 正しいパスで更新
train_data = pd.read_csv(train_data_path)

test_data_path = '/kaggle/input/lmsys-chatbot-arena/test.csv'
test_data = pd.read_csv(test_data_path)

# データの50%をランダムにサンプリングする
sampled_train_data = train_data.sample(frac=0.5, random_state=42)

# 必要に応じてサンプリングしたデータを保存
sampled_train_data_path = '/kaggle/working/sample_train.csv'
sampled_train_data.to_csv(sampled_train_data_path, index=False)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 特徴抽出のためにプロンプトと応答を結合
sampled_train_data['text_a'] = sampled_train_data['prompt'] + " " + sampled_train_data['response_a']
sampled_train_data['text_b'] = sampled_train_data['prompt'] + " " + sampled_train_data['response_b']
test_data['text_a'] = test_data['prompt'] + " " + test_data['response_a']
test_data['text_b'] = test_data['prompt'] + " " + test_data['response_b']

# トークナイザーの初期化
tokenizer = Tokenizer()
# サンプルデータとテストデータを結合し、トークナイザーでフィッティング
tokenizer.fit_on_texts(pd.concat([sampled_train_data['text_a'], sampled_train_data['text_b'], test_data['text_a'], test_data['text_b']]))

# テキストをシーケンスに変換
X_train_a = tokenizer.texts_to_sequences(sampled_train_data['text_a'])
X_train_b = tokenizer.texts_to_sequences(sampled_train_data['text_b'])
X_test_a = tokenizer.texts_to_sequences(test_data['text_a'])
X_test_b = tokenizer.texts_to_sequences(test_data['text_b'])

# シーケンスの長さを揃えるためにパディングを行う
max_length = max(max(len(seq) for seq in X_train_a), max(len(seq) for seq in X_train_b))
X_train_a = pad_sequences(X_train_a, maxlen=max_length, padding='post')  # 末尾にパディング
X_train_b = pad_sequences(X_train_b, maxlen=max_length, padding='post')  # 末尾にパディング
X_test_a = pad_sequences(X_test_a, maxlen=max_length, padding='post')  # 末尾にパディング
X_test_b = pad_sequences(X_test_b, maxlen=max_length, padding='post')  # 末尾にパディング

# ターゲットを抽出
y_train_a = sampled_train_data['winner_model_a']
y_train_b = sampled_train_data['winner_model_b']

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# LSTMモデルを定義
def create_lstm_model(input_length):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=input_length))  # 埋め込み層
    model.add(LSTM(128, return_sequences=True))  # LSTM層（シーケンスを返す）
    model.add(Dropout(0.5))  # 過学習を防ぐためのドロップアウト層
    model.add(LSTM(128))  # もう一つのLSTM層
    model.add(Dense(1, activation='sigmoid'))  # 出力層（シグモイド活性化関数）
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # モデルのコンパイル
    return model

input_length = X_train_a.shape[1]

# text_aのモデルを訓練
model_a = create_lstm_model(input_length)
model_a.fit(X_train_a, y_train_a, epochs=5, batch_size=64, validation_split=0.7)  # 学習の実行

In [ ]:
# text_bのモデルを訓練
model_b = create_lstm_model(input_length)
model_b.fit(X_train_b, y_train_b, epochs=5, batch_size=64, validation_split=0.7)  # 学習の実行

In [ ]:
import numpy as np

# テストセットで予測を行う
test_pred_a = model_a.predict(X_test_a).flatten()  # model_aによる予測を平坦化
test_pred_b = model_b.predict(X_test_b).flatten()  # model_bによる予測を平坦化

# タイの確率を計算（単純化のため均一分布を仮定）
test_pred_tie = np.full(test_pred_a.shape, 1/3)  # 全ての要素を1/3で初期化

In [ ]:
# 提出ファイルを準備
submission = pd.DataFrame({
    'id': test_data['id'],
    'winner_model_a': test_pred_a,
    'winner_model_b': test_pred_b,
    'winner_tie': test_pred_tie
})

# 提出ファイルを保存
submission_path = '/kaggle/working/submission.csv'
submission.to_csv(submission_path, index=False)  # CSV形式で保存

print(f"提出ファイルが {submission_path} に保存されました")